In [1]:
import pandas as pd
import yake

from tqdm import tqdm
tqdm.pandas()

# import swifter

from unidecode import unidecode

In [25]:
from sentence_transformers import SentenceTransformer, util

In [3]:
df = pd.read_csv(r'D:\Work\Data\medium_articles.csv')

In [4]:
kw_extractor = yake.KeywordExtractor()

In [7]:
df = df.head(5000)

In [8]:
df['keywords'] = df['text'].progress_apply(kw_extractor.extract_keywords)

100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [07:26<00:00, 11.20it/s]


In [19]:
df['clean_kw'] = df['keywords'].progress_apply(lambda x: [y[0] for y in x])

100%|██████████████████████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 199980.17it/s]


In [23]:
df_kw = df[['clean_kw']].explode('clean_kw')

In [24]:
df_kw

,clean_kw
0,Josh Riemer
0,Riemer on Unsplash
0,Unsplash Merry Christmas
0,Photo by Josh
0,Unsplash Merry
...,...
4999,Microsoft ’s business
4999,class Visual implements
4999,Power BI Service
4999,tool


In [48]:
#transformer = 'all-mpnet-base-v2'  # provides the best quality
transformer = 'all-MiniLM-L6-v2'  # 5 times faster and still offers good quality

In [34]:
df_kw = df_kw.drop_duplicates()

In [90]:
def cleantext(text):
    text = unidecode(text)
    text = text.lower()
    text = text.strip()
    text = text.replace("'s", "")
    text = text.replace("’s", "")
    text = text.replace("n't", "")
    text = text.replace("'re", "")
    
    return text

In [91]:
df_kw['clean_kw'] = df_kw['clean_kw'].progress_apply(cleantext)

100%|████████████████████████████████████████████████████████████████████████| 51958/51958 [00:00<00:00, 604164.71it/s]


In [92]:
df_kw = df_kw.drop_duplicates()

In [93]:
len(df_kw)

51777

In [159]:
# store the data
cluster_name_list = []
corpus_sentences_list = []
df_all = []

In [160]:
corpus_set = set(df_kw['clean_kw'].head(10000))
# corpus_set = set(df_kw['clean_kw'])
corpus_set_all = corpus_set
cluster = True

In [161]:
len(corpus_set)

10000

In [162]:
cluster_accuracy = 78  # 0-100 (100 = very tight clusters, but higher percentage of no_cluster groups)
min_cluster_size = 5  # set the minimum size of cluster groups. (Lower number = tighter groups)

In [ ]:
# keep looping through until no more clusters are created

cluster_accuracy = cluster_accuracy / 100
model = SentenceTransformer(transformer)#, device='cuda')

while cluster:

    corpus_sentences = list(corpus_set)
    check_len = len(corpus_sentences)

    corpus_embeddings = model.encode(corpus_sentences, batch_size=256, show_progress_bar=True, convert_to_tensor=True)
    clusters = util.community_detection(corpus_embeddings, min_community_size=min_cluster_size, threshold=cluster_accuracy)#, init_max_size=len(corpus_embeddings))

    for keyword, cluster in enumerate(clusters):
        print("\nCluster {}, #{} Elements ".format(keyword + 1, len(cluster)))

        for sentence_id in cluster[0:]:
            print("\t", corpus_sentences[sentence_id])
            corpus_sentences_list.append(corpus_sentences[sentence_id])
            cluster_name_list.append("Cluster {}, #{} Elements ".format(keyword + 1, len(cluster)))

    df_new = pd.DataFrame(None)
    df_new['Cluster Name'] = cluster_name_list
    df_new["Keyword"] = corpus_sentences_list

    df_all.append(df_new)
    have = set(df_new["Keyword"])

    corpus_set = corpus_set_all - have
    remaining = len(corpus_set)
    print("Total Unclustered Keywords: ", remaining)
    if check_len == remaining:
        break

In [224]:
df_new[['Cluster','Num_elements']] = df_new['Cluster Name'].str.split(",", expand=True)

In [123]:
df_new = df_new[['Cluster', 'Keyword', 'Num_elements']]

In [126]:
df_new['len'] = df_new['Keyword'].str.len()

In [144]:
df_new[df_new['Cluster']=='Cluster 54'].sort_values(by='len')

,Cluster,Keyword,Num_elements,len
1980,Cluster 54,facebook,#10 Elements,8
511,Cluster 54,psychology,#7 Elements,10
1988,Cluster 54,social media,#10 Elements,12
1979,Cluster 54,facebook post,#10 Elements,13
1983,Cluster 54,facebook auto,#10 Elements,13
1985,Cluster 54,facebook page,#10 Elements,13
1981,Cluster 54,facebook group,#10 Elements,14
1986,Cluster 54,check facebook,#10 Elements,14
514,Cluster 54,psychology today,#7 Elements,16
1987,Cluster 54,facebook account,#10 Elements,16


In [131]:
df_new.sort_values(by=['Cluster','len'], ascending=[True, True]).tail(75)

,Cluster,Keyword,Num_elements,len
2345,Cluster 94,health concerns,#8 Elements,15
2347,Cluster 94,physical health,#8 Elements,15
761,Cluster 94,habit loop model,#6 Elements,16
762,Cluster 94,habit loop award,#6 Elements,16
763,Cluster 94,habit loop provides,#6 Elements,19
...,...,...,...,...
2381,Cluster 99,partner program story,#8 Elements,21
2385,Cluster 99,medium partner program,#8 Elements,22
2386,Cluster 99,partner program change,#8 Elements,22
2383,Cluster 99,partner program payments,#8 Elements,24


In [132]:
df_new.sort_values(by=['Cluster','len'], ascending=[True, True]).drop_duplicates("Cluster")

,Cluster,Keyword,Num_elements,len
10,Cluster 1,write,#29 Elements,5
1436,Cluster 10,book,#15 Elements,4
2396,Cluster 100,asia,#8 Elements,4
803,Cluster 101,dream,#6 Elements,5
2409,Cluster 102,code,#8 Elements,4
...,...,...,...,...
768,Cluster 95,walk,#6 Elements,4
772,Cluster 96,joel,#6 Elements,4
2369,Cluster 97,adhd,#8 Elements,4
787,Cluster 98,cell,#6 Elements,4


In [164]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [261]:
def name_cluster(cluster_kw_list):
    outlist = []
    for x in cluster_kw_list:
        for y in cluster_kw_list:
#             print(levenshteinDistance(x, y), x, y)
            outlist.append([levenshteinDistance(x, y), x, y])

    df_edit = pd.DataFrame(outlist).rename(columns={0: 'Edit', 1:'Kw1', 2:'Kw2'})
    print(df_edit)
    try:
        df_edit = df_edit[df_edit['Edit']>0]

        sel_index = df_edit[df_edit['Edit'] == df_edit['Edit'].min()].Kw1.str.len().sort_values().index[0]

        cluster_name = df_edit.iloc[sel_index].Kw1
    except:
        print("Whoops")
        cluster_name = df_edit['Kw1'][0]
    return cluster_name

In [185]:
df_edit.loc[df_edit[df_edit['Edit'] == df_edit[df_edit['Edit']>0]['Edit'].min()]['Kw1'].str.len().min()].Kw2

'kanye west fan'

In [227]:
df_new['Cluster'][0]

'Cluster 1'

In [252]:
def namecluster(num):
    abc = df_new[df_new['Cluster'] == f'Cluster {num}']['Keyword'].tolist()
    print(abc)
    return name_cluster(abc)

In [262]:
namecluster(1)

['writing skills', 'writes', 'living writing', 'writing by hand', 'story writing', ' writing', 'write', 'writing', 'writer', 'learned writing', 'writers', 'love writing', 'type of writing', 'writing thing work', 'human writing', 'skill for writing', 'writing thing']
     Edit             Kw1                 Kw2
0       0  writing skills      writing skills
1       9  writing skills              writes
2       9  writing skills      living writing
3       7  writing skills     writing by hand
4      12  writing skills       story writing
..    ...             ...                 ...
284    11   writing thing     type of writing
285     5   writing thing  writing thing work
286    10   writing thing       human writing
287    12   writing thing   skill for writing
288     0   writing thing       writing thing

[289 rows x 3 columns]


'write'

In [245]:
df_new

,Cluster Name,Keyword,Cluster,Num_elements
0,"Cluster 1, #17 Elements",writing skills,Cluster 1,#17 Elements
1,"Cluster 1, #17 Elements",writes,Cluster 1,#17 Elements
2,"Cluster 1, #17 Elements",living writing,Cluster 1,#17 Elements
3,"Cluster 1, #17 Elements",writing by hand,Cluster 1,#17 Elements
4,"Cluster 1, #17 Elements",story writing,Cluster 1,#17 Elements
...,...,...,...,...
965,"Cluster 157, #5 Elements",marketing strategies,Cluster 157,#5 Elements
966,"Cluster 157, #5 Elements",content marketing,Cluster 157,#5 Elements
967,"Cluster 157, #5 Elements",taking marketing,Cluster 157,#5 Elements
968,"Cluster 157, #5 Elements",media marketing,Cluster 157,#5 Elements
